In [ ]:
import weaviate, os

# Connect to a cloud instance of Weaviate (with WCS)
# client = weaviate.connect_to_wcs(
#     cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
#     auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_ADMIN")),
# )

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local()

client.is_ready()

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

# if client.collections.exists("Wikipedia"):
#     print("Collection already exists")
#     inp = input("Do you want to recreate "Wikipedia" collection? Say Y to confirm.")
#     if (inp == "Y"):
#         client.collections.delete("Wikipedia")
#     else:
#         exit

client.collections.delete("Wikipedia")

# Create a collection here - with Cohere as a vectorizer
client.collections.create(
    name="Wikipedia",
    
    vectorizer_config=Configure.Vectorizer.text2vec_cohere(
        model="embed-multilingual-v2.0"
    ),

    # Named Vector example – will be covered in another lesson
    # vectorizer_config=[
    #     Configure.NamedVectors.text2vec_cohere(
    #         name="text_vector",
    #         model="embed-multilingual-v2.0",
    #         source_properties=["text"]
    # )],

    generative_config=Configure.Generative.openai("gpt-4"),

    # properties=[
    #     Property(name="text", data_type=DataType.TEXT),
    #     Property(name="title", data_type=DataType.TEXT, skip_vectorization=True),
    #     Property(name="wiki_id", data_type=DataType.INT, skip_vectorization=True),
    #     Property(name="url", data_type=DataType.TEXT, skip_vectorization=True),
    #     Property(name="lang", data_type=DataType.TEXT, skip_vectorization=True),
    #     Property(name="lang_id", data_type=DataType.INT, skip_vectorization=True),
    #     Property(name="views", data_type=DataType.NUMBER, skip_vectorization=True),
    # ]
)

In [ ]:
from datasets import load_dataset
from tqdm import tqdm

def import_wiki_data(lang, lang_id, max_rows, skip_rows=0):
    print(f"Importing {max_rows} data items for {lang}")

    dataset = load_dataset(f"Cohere/wikipedia-22-12-{lang}-embeddings", split="train", streaming=True)
    dataset = dataset.skip(skip_rows)

    # counter = 0
    counter = skip_rows

    wikipedia = client.collections.get("Wikipedia")

    with wikipedia.batch.fixed_size(batch_size=1000, concurrent_requests=4) as batch:
        for item in tqdm(dataset, initial=skip_rows, total=max_rows):
            vector = item["emb"]
            data_to_insert = {   
                "text": item["text"],
                "wiki_id": item["wiki_id"],
                "title": item["title"],
                "url": item["url"],
                "views": item["views"],
                "lang": lang,
                "lang_id": lang_id,
            }

            batch.add_object(
                properties=data_to_insert,
                vector=vector
                
                # Named Vector example – will be covered in another lesson
                # vector={
                #     "text_vector": vector   
                # }
            )
            
            counter += 1
            if counter >= max_rows:
                break

            if (counter % 200 == 0):
                # this doesn't seem to see the errors, that are later visible in the final check
                if (len(wikipedia.batch.failed_objects)>0):
                    print(f"Some errors {len(wikipedia.batch.failed_objects)}")
                    print(wikipedia.batch.failed_objects[-1])
    
    # check for errors at the end
    if (len(wikipedia.batch.failed_objects)>0):
        print("Final error check")
        print(f"Some errors {len(wikipedia.batch.failed_objects)}")
        print(wikipedia.batch.failed_objects[-1])
    
    print(f"Imported {counter} items for {lang}")
    print("-----------------------------------")

In [ ]:
import_per_country = 10_000

import_wiki_data("en", 0, import_per_country, 0)
import_wiki_data("de", 1, import_per_country, 0)
import_wiki_data("fr", 2, import_per_country, 0)
# import_wiki_data("es", 3, import_per_country, 0)
# import_wiki_data("it", 4, import_per_country, 0)